In [1]:
import torch

from gfos.model.gnn import LayoutModel
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.data.utils import load_layout

In [3]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"

SOURCE = "xla"
SEARCH = "random"

In [4]:
normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["test"],
    normalizer=normalizer,
)

Loading data:   0%|          | 0/8 [00:00<?, ?it/s, 05ae41e26dd3c4c06390371a0423233c]g:\projects\gfos\src\gfos\data\dataset.py:280: RuntimeWarning: invalid value encountered in divide
  runtime = (runtime - mu) / std
Loading data: 100%|██████████| 8/8 [00:04<00:00,  1.88it/s, fbaa8bb6a1aed9988281085c91065c05]


In [11]:
model = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_xla_random_20231019_155806\runs\2023-10-19_15-58-06\wandb\run-20231019_155948-991bvbsj\files\319_0.6165.pth")
model.load_state_dict(state_dict["state_dict"])

Disable config edge weight


<All keys matched successfully>

In [12]:
model.eval().cuda()

LayoutModel(
  (embedding): Embedding(120, 32)
  (node_gnn): Sequential(
    (0): SAGEConv(144, 64, aggr=mean)
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): SAGEConv(64, 64, aggr=mean)
    (3): LeakyReLU(negative_slope=0.01, inplace=True)
    (4): SAGEConv(64, 64, aggr=mean)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): SAGEConv(64, 64, aggr=mean)
    (7): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (config_neighbor_gnn): Sequential(
    (0): GATConv(64, 64, heads=1)
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): GATConv(64, 64, heads=1)
    (3): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (config_gnn): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): SAGEConv(192, 64, aggr=mean)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): SAGEConv(64, 64, aggr=mean)
    (4): LeakyReLU(negative_slope=0.01, inplace=True)
    (5): SAGEConv(64, 64, aggr=mean)
    (6): LeakyReLU(negative_slope=0.01, inplace=True)


In [13]:
device = "cuda"
batch_size = 100

results = {}

with torch.no_grad():
    for record in test_dataset:
        model_id = record["model_id"]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]
        config_edge_index = record["config_edge_index"]
        config_edge_weight = record["config_edge_weight"]
        config_edge_path = record["config_edge_path"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edge_index,
            config_edge_weight,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edge_index.to(device),
            config_edge_weight.to(device),
        )

        c = len(config_runtime)
        outs = []

        for i in range(0, c, batch_size):
            end_i = min(i + batch_size, c)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i:end_i],
                node_config_ids,
                config_edge_index,
                config_edge_weight,
            )
            outs.append(out.detach().cpu())
        results[model_id] = torch.concat(outs)

In [15]:
import pickle


pickle.dump(results, open("results.pkl", "wb"))